In [1]:
import os

from pyspark.sql import SparkSession
import logging

from utils import read_geo, read_events, add_closest_city

In [2]:
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["YARN_CONF_DIR"] = "/etc/hadoop/conf"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
os.environ["HADOOP_CONF_DIR"] = "/etc/hadoop/conf"

In [3]:
# DEV Constants
GEO_DIR = "/user/solovyovyu/geo.csv"
EVENTS_DIR = "/user/solovyovyu/data/geo/events"
OUT_PATH = "/user/solovyovyu/analytics"

# PROD Constants
# EVENTS_DIR = "/user/master/data/geo/events"

In [4]:
def setup_logging():
    """Configure logging"""
    logger = logging.getLogger(__name__)
    logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
    return logger

logger = setup_logging()

In [5]:
try:
    spark = SparkSession.builder \
        .master("local") \
        .appName("Mart User") \
        .getOrCreate()
    logger.info("SparkSession создан успешно.")
except Exception as e:
    logger.error(f"Ошибка при создании SparkSession: {e}")
    raise

25/02/09 10:11:48 WARN Utils: Your hostname, fhm0sehqe09hbdufcek4 resolves to a loopback address: 127.0.1.1; using 172.16.0.5 instead (on interface eth0)
25/02/09 10:11:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/09 10:11:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2025-02-09 10:11:51,719 - INFO - SparkSession создан успешно.


In [6]:
geo_df = read_geo(GEO_DIR, spark, logger)

2025-02-09 10:12:01,402 - INFO - Geo data is read from /user/solovyovyu/geo.csv. Number of rows: 24


In [7]:
messages_df = read_events("message", EVENTS_DIR, spark, logger)

2025-02-09 10:12:03,538 - INFO - Events message are read from /user/solovyovyu/data/geo/events. Number of rows: 5395


In [8]:
reaction_df = read_events("reaction", EVENTS_DIR, spark, logger)

2025-02-09 10:12:04,085 - INFO - Events reaction are read from /user/solovyovyu/data/geo/events. Number of rows: 362963


In [9]:
subscription_df = read_events("subscription", EVENTS_DIR, spark, logger)

2025-02-09 10:12:04,698 - INFO - Events subscription are read from /user/solovyovyu/data/geo/events. Number of rows: 693990


In [10]:
# 
message_with_distance = add_closest_city(messages_df, geo_df, spark, logger)

message_with_distance.show()

25/02/09 10:12:07 ERROR Executor: Exception in task 0.0 in stage 17.0 (TID 13)1]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/lessons/utils.py", line 72, in calculate_distance
    d_lat = F.radians(lat2 - lat1)
  File "/usr/local/lib/python3.8/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 174, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 3395, in radians
    return _invoke_function_over_columns("radians", col)
  File "/usr/local/lib/python3.8/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 105, in _invoke_function_over_columns
    return _invoke_function(name, *(_to_java_column(col) for col in cols))
  File "/usr/local/lib/python3.8/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 105, in <genexpr>
    return _invoke_function(name, *(_to_java_column(col) f

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/lessons/utils.py", line 72, in calculate_distance
    d_lat = F.radians(lat2 - lat1)
  File "/usr/local/lib/python3.8/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 174, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 3395, in radians
    return _invoke_function_over_columns("radians", col)
  File "/usr/local/lib/python3.8/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 105, in _invoke_function_over_columns
    return _invoke_function(name, *(_to_java_column(col) for col in cols))
  File "/usr/local/lib/python3.8/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 105, in <genexpr>
    return _invoke_function(name, *(_to_java_column(col) for col in cols))
  File 